In [359]:
import random
import pygame
import numpy as np

random_seed = 42

Initialization

In [327]:
def create_initiate_state(n: int):
    all_possible_positions = [(i, j) for i in range(n) for j in range(n)]
    sample = random.sample(all_possible_positions, n)
    return set(sample)


In [330]:
def get_matrix_view(positions: set, n: int):
    matrix = np.zeros((n, n), dtype=int)
    for i, j in positions:
        matrix[i][j] = 1
    return matrix

In [332]:
random.seed(random_seed)
queens = create_initiate_state(5)
get_matrix_view(queens, 5)

array([[1, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0]])

In [333]:
def get_state_neighbors(queen: tuple[int, int], n: int, depth: int, obstacles: set | None = None):
    neighbors = set()
    q_row, q_col = queen
    horizontal_directions = [(0, 1), (0, -1)]
    vertical_directions = [(1, 0), (-1, 0)]
    diag_directions = [(1, 1), (-1, 1), (1, -1), (-1, -1)]
    directions = horizontal_directions + vertical_directions + diag_directions
    for direction in directions:
        d_row, d_col = direction
        c_row, c_col = q_row + d_row, q_col + d_col
        current_depth = 1
        while 0 <= c_row < n and 0 <= c_col < n and current_depth <= depth:
            if obstacles and (c_row, c_col) in obstacles:
                break
            neighbors.add((c_row, c_col))
            c_row, c_col = c_row + d_row, c_col + d_col
            current_depth += 1
    return neighbors

In [334]:
def heuristic(queens: set):
    h = 0
    n = len(queens)
    remaining_queens: set = queens.copy()
    while len(remaining_queens) > 0:
        queen = remaining_queens.pop()
        neighbors = get_state_neighbors(queen, n, depth=n)
        h += len(neighbors & remaining_queens)
    return h


In [335]:
heuristic(queens)

7

In [336]:
def get_all_neighbor_states(queens: set, depth: int):
    n = len(queens)
    neighbors_states = list()
    for queen in queens:
        remaining_queens = queens - {queen}
        neighbors = get_state_neighbors(queen, n, depth, remaining_queens)
        for neighbor in neighbors:
            neighbors_states.append(remaining_queens | {neighbor})
    return neighbors_states


In [337]:
def get_best_state(states: list):
    best_state = states.pop()
    best_h = heuristic(best_state)
    for state in states:
        h = heuristic(state)
        if h < best_h:
            best_h = h
            best_state = state
    return best_state, best_h

In [338]:
def hill_climbing(initial_state, depth: int, side_way_moves: int = 0, ):
    current_state, current_h = initial_state, heuristic(initial_state)
    current_side_way_moves = 0
    while True:
        all_neighbors = get_all_neighbor_states(current_state, depth)
        best_neighbor_state, best_neighbor_h = get_best_state(all_neighbors)
        if best_neighbor_h == current_h:
            if current_side_way_moves < side_way_moves:
                current_side_way_moves += 1
            else:
                return current_state, current_h
        elif best_neighbor_h > current_h:
            return current_state, current_h
        current_state, current_h = best_neighbor_state, best_neighbor_h


In [339]:
def n_queens(n: int, restarts: int = 0, side_way_moves: int = 0, depth: int | None = None, rd_seed: int | None = None):
    if depth is None:
        depth = n
    elif depth < 1:
        raise ValueError("depth must be at least 1")
    if rd_seed:
        random.seed(rd_seed)
    else:
        random.seed()

    initial_state = create_initiate_state(n)
    best_state, best_h = hill_climbing(initial_state, depth, side_way_moves)
    if best_h == 0:
        return best_state, best_h
    for restart in range(restarts + 1):
        initial_state = create_initiate_state(n)
        current_best_state, current_best_h = hill_climbing(initial_state, side_way_moves)
        if current_best_h < best_h:
            best_state, best_h = current_best_state, current_best_h
        if current_best_h == 0:
            break
    return best_state, best_h

In [357]:
n = 8
best_state, best_h = n_queens(n, restarts=10, side_way_moves=10)
print(best_h)
print(get_matrix_view(best_state, n))

0
[[0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]]


# Visualization

In [370]:
TILE_SIZE = 20
FONT_SIZE = 20
FONT_COLOR = (0, 0, 0)
BLACK_TILE_COLOR = (0, 0, 0)
WHITE_TILE_COLOR = (255, 255, 255)
QUEEN_COLOR = (255, 0, 0)
STEP_INTERVAL = 25  # in milliseconds


In [376]:
def draw_tile(surface, position, color):
    x, y = position
    pygame.draw.rect(surface=surface, color=color,
                     rect=(x * TILE_SIZE, y * TILE_SIZE, TILE_SIZE, TILE_SIZE))


def draw_text(position, font, screen, text, color):
    img_expanded = font.render(text, True, color)
    screen.blit(img_expanded, position)

In [377]:
def run_visualization(n: int, rd_seed: int | None = None):
    pygame.init()
    screen = pygame.display.set_mode((n * TILE_SIZE, n * TILE_SIZE + 4 * FONT_SIZE))
    font = pygame.font.Font("roboto.ttf", FONT_SIZE)

    screen.fill("gray")
    pygame.display.flip()

    for i in range(n):
        for j in range(n):
            color = WHITE_TILE_COLOR
            if (i + j) % 2 == 0:
                color = BLACK_TILE_COLOR
            draw_tile(screen, (i, j), color)

    pygame.display.flip()

    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
    pygame.quit()

In [378]:
run_visualization(42)